In [2]:
import yfinance as yf
from pandas_datareader import data as web
import datetime

#なんのデータを得るかを入力
tickers = ['AAPL']

#取得開始日を入力
start = "2022-01-02"

#取得終了日を入力
end = datetime.date.today()

#Yahoofinanceから取得するように設定
yf.pdr_override()

#データの取得を実行
data = web.get_data_yahoo(tickers, start, end)

[*********************100%%**********************]  1 of 1 completed


In [1]:
import talib as ta
#talibを使えば様々なテクニカル指標が計算できる、例としてMACD

def MACD(close, n1, n2, ns):
    macd, macdsignal, macdhist = ta.MACD(close, fastperiod=n1, slowperiod=n2, signalperiod=ns)
    return macd, macdsignal


In [3]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA

class MACDCross(Strategy):
    n1 = 12 #短期EMAの期間
    n2 = 26 #長期EMAの期間
    ns = 9 #シグナル（MACDのSMA）の期間

    def init(self):
        self.macd, self.macdsignal = self.I(MACD, self.data.Close, self.n1, self.n2, self.ns)

    def next(self): # チャートデータの行ごとに呼び出される
        if crossover(self.macd, self.macdsignal): #macdがsignalを上回った時
            self.buy() # 買い
        elif crossover(self.macdsignal, self.macd): #signalがmacdを上回った時
            self.position.close() # 売り


/home/shunta/py_venvs/py1/.venv1/lib/python3.11/site-packages/backtesting/test/__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),
/home/shunta/py_venvs/py1/.venv1/lib/python3.11/site-packages/backtesting/test/__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),


In [4]:
# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    MACDCross, # 売買戦略
    cash=1000, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=True, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
bt.plot() # 実行結果（グラフ）


Start                     2022-01-03 00:00:00
End                       2024-05-03 00:00:00
Duration                    851 days 00:00:00
Exposure Time [%]                   47.018739
Equity Final [$]                  1065.530068
Equity Peak [$]                   1201.980177
Return [%]                           6.553007
Buy & Hold Return [%]                0.752712
Return (Ann.) [%]                    2.762344
Volatility (Ann.) [%]               15.869163
Sharpe Ratio                          0.17407
Sortino Ratio                         0.26482
Calmar Ratio                          0.16873
Max. Drawdown [%]                   -16.37137
Avg. Drawdown [%]                   -7.684626
Max. Drawdown Duration      241 days 00:00:00
Avg. Drawdown Duration       95 days 00:00:00
# Trades                                   21
Win Rate [%]                        33.333333
Best Trade [%]                      20.593488
Worst Trade [%]                    -12.302583
Avg. Trade [%]                    

Row(id='1494', ...)

gio: file:///home/shunta/py_venvs/py1/投資/BackTest/MACDCross.html: No application is registered as handling this file


In [5]:
#最適化
output2=bt.optimize(
    n1=range(10, 100, 10),
    n2=range(10, 300, 10),
    ns=range(10, 50, 5), 
    maximize='Equity Final [$]',
    method='grid'
)

print(output2)
bt.plot()


/home/shunta/py_venvs/py1/.venv1/lib/python3.11/site-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 2088 configurations.
  output = _optimize_grid()


Start                     2022-01-03 00:00:00
End                       2024-05-03 00:00:00
Duration                    851 days 00:00:00
Exposure Time [%]                   29.642249
Equity Final [$]                  1373.700296
Equity Peak [$]                   1405.282817
Return [%]                           37.37003
Buy & Hold Return [%]                0.752712
Return (Ann.) [%]                   14.603332
Volatility (Ann.) [%]               12.742516
Sharpe Ratio                         1.146032
Sortino Ratio                        2.259631
Calmar Ratio                         1.850194
Max. Drawdown [%]                   -7.892863
Avg. Drawdown [%]                   -1.928032
Max. Drawdown Duration      277 days 00:00:00
Avg. Drawdown Duration       20 days 00:00:00
# Trades                                    2
Win Rate [%]                            100.0
Best Trade [%]                      31.350982
Worst Trade [%]                      0.247392
Avg. Trade [%]                    

Row(id='2716', ...)

gio: file:///home/shunta/py_venvs/py1/投資/BackTest/MACDCross(n1=20,n2=210,ns=20).html: No application is registered as handling this file
